In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

print("Erfolgreich")

Erfolgreich


In [24]:
df = pd.read_csv('labels.csv')
df

,filepath,xmin,xmax,ymin,ymax
0,./images/Cars213.xml,150,233,160,194
1,./images/Cars207.xml,254,372,229,264
2,./images/Cars398.xml,72,196,139,222
3,./images/Cars367.xml,224,382,161,222
4,./images/N107.xml,207,356,174,287
...,...,...,...,...,...
653,./images/Cars381.xml,65,147,131,155
654,./images/Cars222.xml,178,235,158,170
655,./images/Cars236.xml,223,250,106,119
656,./images/N240.xml,164,285,99,137


In [25]:
import xml.etree.ElementTree as xet

In [26]:
filename = df['filepath'][0]
filename

'./images/Cars213.xml'

In [27]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [28]:
getFilename(filename)

'./images/Cars213.png'

In [29]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images/Cars213.png',
 './images/Cars207.png',
 './images/Cars398.png',
 './images/Cars367.png',
 './images/N107.jpeg',
 './images/Cars401.png',
 './images/Cars415.png',
 './images/N113.jpeg',
 './images/Cars373.png',
 './images/Cars429.png',
 './images/N62.jpeg',
 './images/Cars47.png',
 './images/Cars53.png',
 './images/Cars165.png',
 './images/Cars171.png',
 './images/N89.jpeg',
 './images/Cars159.png',
 './images/Cars84.png',
 './images/Cars90.png',
 './images/Cars91.png',
 './images/Cars85.png',
 './images/Cars158.png',
 './images/N88.jpeg',
 './images/Cars170.png',
 './images/Cars164.png',
 './images/Cars52.png',
 './images/Cars46.png',
 './images/N63.jpeg',
 './images/N77.jpeg',
 './images/Cars428.png',
 './images/Cars414.png',
 './images/Cars372.png',
 './images/N112.jpeg',
 './images/N106.jpeg',
 './images/Cars366.png',
 './images/Cars400.png',
 './images/Cars399.png',
 './images/Cars206.png',
 './images/Cars212.png',
 './images/Cars204.png',
 './images/Cars210.png',
 './im

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./images/Cars213.png'

In [ ]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 150	233	160	194
cv2.rectangle(img,(150,160),(233,194),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [30]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

print("Erfolgreich")

Erfolgreich


In [31]:
labels = df.iloc[:,1:].values

In [32]:
print(labels)

[[ 150  233  160  194]
 [ 254  372  229  264]
 [  72  196  139  222]
 ...
 [ 223  250  106  119]
 [ 164  285   99  137]
 [1804 2493 1734 1882]]


In [33]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [34]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [35]:
X.shape,y.shape

((658, 224, 224, 3), (658, 4))

In [36]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((526, 224, 224, 3), (132, 224, 224, 3), (526, 4), (132, 4))

### Deep Learning Model

In [37]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

print("Erfolgreich")

Erfolgreich


In [38]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=True
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [39]:
# compile model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 111, 111, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 111, 111, 32) 0           batch_normalization_203[0][0]    
____________________________________________________________________________________________

batch_normalization_360 (BatchN (None, 5, 5, 384)    1152        conv2d_360[0][0]                 
__________________________________________________________________________________________________
batch_normalization_362 (BatchN (None, 5, 5, 288)    864         conv2d_362[0][0]                 
__________________________________________________________________________________________________
batch_normalization_365 (BatchN (None, 5, 5, 320)    960         conv2d_365[0][0]                 
__________________________________________________________________________________________________
activation_360 (Activation)     (None, 5, 5, 384)    0           batch_normalization_360[0][0]    
__________________________________________________________________________________________________
activation_362 (Activation)     (None, 5, 5, 288)    0           batch_normalization_362[0][0]    
__________________________________________________________________________________________________
activation

### model training

In [40]:
from tensorflow.keras.callbacks import TensorBoard
print("Erfolgreich")

Erfolgreich


In [41]:
tfb = TensorBoard('object_detection')

In [43]:
history = model.fit(x=x_train,y=y_train,batch_size=15,epochs=300,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/300
36/36 [==============================] - 262s 7s/step - loss: 0.0310 - accuracy: 0.6084 - val_loss: 0.0216 - val_accuracy: 0.6894
Epoch 2/300
36/36 [==============================] - 273s 8s/step - loss: 0.0128 - accuracy: 0.7129 - val_loss: 0.0151 - val_accuracy: 0.5833
Epoch 3/300
36/36 [==============================] - 288s 8s/step - loss: 0.0048 - accuracy: 0.8251 - val_loss: 0.0113 - val_accuracy: 0.7500
Epoch 4/300
36/36 [==============================] - 262s 7s/step - loss: 0.0040 - accuracy: 0.8631 - val_loss: 0.0120 - val_accuracy: 0.6439
Epoch 5/300
36/36 [==============================] - 283s 8s/step - loss: 0.0040 - accuracy: 0.8783 - val_loss: 0.0109 - val_accuracy: 0.8333
Epoch 6/300
36/36 [==============================] - 262s 7s/step - loss: 0.0050 - accuracy: 0.8802 - val_loss: 0.0264 - val_accuracy: 0.5530
Epoch 7/300
36/36 [==============================] - 278s 8s/step - loss: 0.0028 - accuracy: 0.9106 - val_loss: 0.0146 - val_accuracy: 0.8409
Epoch 

KeyboardInterrupt: 

In [26]:
history = model.fit(x=x_train,y=y_train,batch_size=15,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
24/24 [==============================] - 44s 2s/step - loss: 0.0014 - val_loss: 0.0106
Epoch 103/200
24/24 [==============================] - 44s 2s/step - loss: 0.0012 - val_loss: 0.0105
Epoch 104/200
24/24 [==============================] - 46s 2s/step - loss: 9.4261e-04 - val_loss: 0.0108
Epoch 105/200
24/24 [==============================] - 43s 2s/step - loss: 9.8487e-04 - val_loss: 0.0107
Epoch 106/200
24/24 [==============================] - 43s 2s/step - loss: 9.2338e-04 - val_loss: 0.0104
Epoch 107/200
24/24 [==============================] - 44s 2s/step - loss: 8.5638e-04 - val_loss: 0.0111
Epoch 108/200
24/24 [==============================] - 44s 2s/step - loss: 7.8602e-04 - val_loss: 0.0105
Epoch 109/200
24/24 [==============================] - 43s 2s/step - loss: 8.2099e-04 - val_loss: 0.0107
Epoch 110/200
24/24 [==============================] - 43s 2s/step - loss: 7.9119e-04 - val_loss: 0.0123
Epoch 111/200
24/24 [==============================] - 43s 2s/s

24/24 [==============================] - 43s 2s/step - loss: 0.0010 - val_loss: 0.0103
Epoch 183/200
24/24 [==============================] - 43s 2s/step - loss: 9.4176e-04 - val_loss: 0.0103
Epoch 184/200
24/24 [==============================] - 43s 2s/step - loss: 9.3827e-04 - val_loss: 0.0099
Epoch 185/200
24/24 [==============================] - 44s 2s/step - loss: 9.3869e-04 - val_loss: 0.0102
Epoch 186/200
24/24 [==============================] - 44s 2s/step - loss: 8.9185e-04 - val_loss: 0.0098
Epoch 187/200
24/24 [==============================] - 43s 2s/step - loss: 8.2720e-04 - val_loss: 0.0099
Epoch 188/200
24/24 [==============================] - 43s 2s/step - loss: 9.5261e-04 - val_loss: 0.0104
Epoch 189/200
24/24 [==============================] - 43s 2s/step - loss: 9.2919e-04 - val_loss: 0.0100
Epoch 190/200
24/24 [==============================] - 43s 2s/step - loss: 9.2255e-04 - val_loss: 0.0100
Epoch 191/200
24/24 [==============================] - 43s 2s/step - loss

In [27]:
model.save('./models/object_detection.h5')